In [1]:
import glob
import tensorflow as tf
import datetime
import matplotlib.pyplot as plt
import seaborn as sb
import segmentation_models as sm
import functools as ft
import pandas as pd

# disable GPU computation
# tf.config.set_visible_devices([], 'GPU')

sm.set_framework('tf.keras')
sm.framework()

tf.random.set_seed(1024)
SEED = 100

Segmentation Models: using `keras` framework.


In [2]:
def get_image_decode(image, label):
    image = tf.io.read_file(image, "image")
    label = tf.io.read_file(label, "label")

    image = tf.image.decode_image(image)
    label = tf.image.decode_image(label)

    return image, label


# [w, h, c], 448, 448, 3
def decode_crop(image, label):
    image = image[368 // 2 : -(368 // 2), 256 // 2 : -(256 // 2)]
    label = label[368 // 2 : -(368 // 2), 256 // 2 : -(256 // 2)]

    img_array = []
    label_array = []

    for index in range(4 * 8):
        x, y = index // 8, index % 8
        img_array.append(image[448 * x : 448 * (1 + x), 448 * y : 448 * (1 + y)])
        label_array.append(label[448 * x : 448 * (1 + x), 448 * y : 448 * (1 + y)])

    return tf.data.Dataset.from_tensor_slices((img_array, label_array))


def get_mask(image, label):
    labels = []
    labels.append((label[:, :, 0] == 0) & (label[:, :, 1] == 0) & (label[:, :, 2] == 0))
    labels.append(
        (label[:, :, 0] == 128) & (label[:, :, 1] == 0) & (label[:, :, 2] == 0)
    )
    labels.append(
        (label[:, :, 0] == 128) & (label[:, :, 1] == 64) & (label[:, :, 2] == 128)
    )
    labels.append(
        (label[:, :, 0] == 0) & (label[:, :, 1] == 128) & (label[:, :, 2] == 0)
    )
    labels.append(
        (label[:, :, 0] == 128) & (label[:, :, 1] == 128) & (label[:, :, 2] == 0)
    )
    labels.append(
        (label[:, :, 0] == 64) & (label[:, :, 1] == 0) & (label[:, :, 2] == 128)
    )
    labels.append(
        (label[:, :, 0] == 192) & (label[:, :, 1] == 0) & (label[:, :, 2] == 192)
    )
    labels.append(
        (label[:, :, 0] == 64) & (label[:, :, 1] == 64) & (label[:, :, 2] == 0)
    )
    labels = tf.cast(labels, tf.float32)
    image = tf.cast(image, tf.float32)

    # must perform this
    return image, tf.transpose(labels, [1, 2, 0])


def create_ds(batch_size, istrain=True, maximage=False):
    AUTOTUNE = tf.data.AUTOTUNE

    if istrain:
        directory = "/home/hackerton/Downloads/uavid_v1.5_official_release/uavid_train/**/Images/*.png"
        images = glob.glob(directory, recursive=True)
        directory = "/home/hackerton/Downloads/uavid_v1.5_official_release/uavid_train/**/Labels/*.png"
        labels = glob.glob(directory, recursive=True)
    else:
        directory = "/home/hackerton/Downloads/uavid_v1.5_official_release/uavid_val/**/Images/*.png"
        images = glob.glob(directory, recursive=True)
        directory = "/home/hackerton/Downloads/uavid_v1.5_official_release/uavid_val/**/Labels/*.png"
        labels = glob.glob(directory, recursive=True)

    ds = tf.data.Dataset.from_tensor_slices((images, labels))
    ds = ds.cache()
    if istrain:
        ds = ds.shuffle(6400, SEED, reshuffle_each_iteration=True)
    ds = ds.map(get_image_decode, AUTOTUNE)

    if not maximage:
        ds = ds.flat_map(decode_crop)

    ds = ds.map(get_mask, AUTOTUNE)

    # batch and prefetch
    ds = ds.batch(batch_size)

    if istrain:
        ds = ds.prefetch(AUTOTUNE)

    return ds

In [3]:
# data = create_ds(1, False, True)

# total_mask_sum = tf.zeros([8], tf.int64)
# for image, mask in data:
#     mask = tf.cast(mask, tf.int64)
#     mask_sum = tf.reduce_sum(mask, [0, 1, 2])

#     total_mask_sum += mask_sum

In [4]:
# plt.figure(figsize=(15, 7))
# plt.bar([str(i) for i in range(8)], total_mask_sum / tf.reduce_max(total_mask_sum), color=['blue', 'red', 'green', 'yellow'], align='center', alpha=0.8)
# plt.grid(True)
# plt.title('The number of samples mask in each class')

# (total_mask_sum / tf.reduce_mean(total_mask_sum))

In [5]:
def create_backbone():
    _backbone = tf.keras.applications.EfficientNetB0(
        include_top=False, input_shape=[None, None, 3]
    )

    outputs = [
        layer.output
        for layer in _backbone.layers
        if layer.name
        in [
            "block2a_activation",
            "block3a_activation",
            "block5a_activation",
            "block7a_activation",
        ]
    ]

    return tf.keras.Model(
        inputs=[_backbone.input], outputs=outputs, name="efficientb0_backbone"
    )


class FPN(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(name="Feature_Pyramid_Network", **kwargs)

        self.backbone = create_backbone()
        self.conv5_1x1 = tf.keras.layers.Conv2D(
            filters=256, kernel_size=(1, 1), padding="same"
        )
        self.conv4_1x1 = tf.keras.layers.Conv2D(
            filters=256, kernel_size=(1, 1), padding="same"
        )
        self.conv3_1x1 = tf.keras.layers.Conv2D(
            filters=256, kernel_size=(1, 1), padding="same"
        )
        self.conv2_1x1 = tf.keras.layers.Conv2D(
            filters=256, kernel_size=(1, 1), padding="same"
        )
        self.conv5_3x3_1 = tf.keras.layers.Conv2D(
            filters=128, kernel_size=(3, 3), padding="same", activation="relu"
        )
        self.conv5_3x3_2 = tf.keras.layers.Conv2D(
            filters=128, kernel_size=(3, 3), padding="same", activation="relu"
        )
        self.conv5_bn = tf.keras.layers.BatchNormalization()
        self.conv4_3x3_1 = tf.keras.layers.Conv2D(
            filters=128, kernel_size=(3, 3), padding="same", activation="relu"
        )
        self.conv4_3x3_2 = tf.keras.layers.Conv2D(
            filters=128, kernel_size=(3, 3), padding="same", activation="relu"
        )
        self.conv4_bn = tf.keras.layers.BatchNormalization()
        self.conv3_3x3_1 = tf.keras.layers.Conv2D(
            filters=128, kernel_size=(3, 3), padding="same", activation="relu"
        )
        self.conv3_3x3_2 = tf.keras.layers.Conv2D(
            filters=128, kernel_size=(3, 3), padding="same", activation="relu"
        )
        self.conv3_bn = tf.keras.layers.BatchNormalization()
        self.conv2_3x3_1 = tf.keras.layers.Conv2D(
            filters=128, kernel_size=(3, 3), padding="same", activation="relu"
        )
        self.conv2_3x3_2 = tf.keras.layers.Conv2D(
            filters=128, kernel_size=(3, 3), padding="same", activation="relu"
        )
        self.conv2_bn = tf.keras.layers.BatchNormalization()
        self.upscale = tf.keras.layers.UpSampling2D(size=(2, 2))

    def call(self, images, training=False):
        # 112x112, 56x56, 28x28, 14x14
        conv2, conv3, conv4, conv5 = self.backbone(images, training=training)
        conv5_m = self.conv5_1x1(conv5)
        conv5_p = self.conv5_3x3_1(conv5_m)
        conv5_p = self.conv5_3x3_2(conv5_p)
        conv5_p = self.conv5_bn(conv5_p, training=training)

        conv4_m_1 = self.upscale(conv5_m)
        conv4_m_2 = self.conv4_1x1(conv4)
        conv4_m = conv4_m_1 + conv4_m_2
        conv4_p = self.conv4_3x3_1(conv4_m)
        conv4_p = self.conv4_3x3_2(conv4_p)
        conv4_p = self.conv4_bn(conv4_p, training=training)

        conv3_m_1 = self.upscale(conv4_m)
        conv3_m_2 = self.conv3_1x1(conv3)
        conv3_m = conv3_m_1 + conv3_m_2
        conv3_p = self.conv3_3x3_1(conv3_m)
        conv3_p = self.conv3_3x3_2(conv3_p)
        conv3_p = self.conv3_bn(conv3_p, training=training)

        conv2_m_1 = self.upscale(conv3_m)
        conv2_m_2 = self.conv2_1x1(conv2)
        conv2_m = conv2_m_1 + conv2_m_2
        conv2_p = self.conv2_3x3_1(conv2_m)
        conv2_p = self.conv2_3x3_2(conv2_p)
        conv2_p = self.conv2_bn(conv2_p, training=training)
        return conv5_p, conv4_p, conv3_p, conv2_p


class FCN(tf.keras.Model):
    def __init__(self, n_classes=8, **kwargs):
        super().__init__(name="FCN", **kwargs)
        self.fpn = FPN()
        self.upscale_2x = tf.keras.layers.UpSampling2D()
        self.upscale_4x = tf.keras.layers.UpSampling2D((4, 4))
        self.upscale_8x = tf.keras.layers.UpSampling2D((8, 8))
        self.concat = tf.keras.layers.Concatenate()
        self.conv6 = tf.keras.layers.Conv2D(
            filters=(512), kernel_size=(3, 3), padding="same", activation="relu"
        )
        self.bnorm = tf.keras.layers.BatchNormalization()
        self.conv7 = tf.keras.layers.Conv2D(
            filters=n_classes, kernel_size=(1, 1), padding="same", activation="relu"
        )
        self.upscale_final = tf.keras.layers.UpSampling2D(
            size=(4, 4), interpolation="bilinear"
        )
        self.final_activation = tf.keras.layers.Activation("softmax")

    def call(self, images, training=False):
        conv5_p, conv4_p, conv3_p, conv2_p = self.fpn(images, training=training)
        m_5 = self.upscale_8x(conv5_p)
        m_4 = self.upscale_4x(conv4_p)
        m_3 = self.upscale_2x(conv3_p)
        m_2 = conv2_p

        m_all = self.concat([m_2, m_3, m_4, m_5])
        m_all = self.conv6(m_all)
        m_all = self.bnorm(m_all, training=training)
        m_all = self.conv7(m_all)
        m_all = self.upscale_final(m_all)
        m_all = self.final_activation(m_all)

        return m_all


class FCN_ORIG(tf.keras.Model):
    def __init__(self, n_classes=8, **kwargs):
        super().__init__(name="FCN_ORIG", **kwargs)

        self.backbone = create_backbone()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=(n_classes), kernel_size=(1, 1), padding="same", activation="relu"
        )
        self.conv2 = tf.keras.layers.Conv2D(
            filters=(n_classes), kernel_size=(1, 1), padding="same", activation="relu"
        )
        self.conv3 = tf.keras.layers.Conv2D(
            filters=(n_classes), kernel_size=(1, 1), padding="same", activation="relu"
        )
        self.upscale2x_1 = tf.keras.layers.Convolution2DTranspose(
            filters=8,
            kernel_size=(4, 4),
            strides=(2, 2),
            padding="same",
            activation="relu",
        )
        self.upscale2x_2 = tf.keras.layers.Convolution2DTranspose(
            filters=8,
            kernel_size=(4, 4),
            strides=(2, 2),
            padding="same",
            activation="relu",
        )
        self.upscale8x = tf.keras.layers.Convolution2DTranspose(
            filters=8,
            kernel_size=(16, 16),
            strides=(8, 8),
            padding="same",
            activation="relu",
        )
        self.final_activation = tf.keras.layers.Activation("softmax")

    def call(self, images, training=False):
        _, conv1_o, conv2_o, conv3_o = self.backbone(images, training=training)
        conv1_o = self.conv1(conv1_o)
        conv2_o = self.conv2(conv2_o)
        conv3_o = self.conv3(conv3_o)

        fcn_16x = self.upscale2x_1(conv3_o) + conv2_o
        fcn_8x = self.upscale2x_2(fcn_16x) + conv1_o
        final_output = self.upscale8x(fcn_8x)
        final_output = self.final_activation(final_output)
        return final_output


In [6]:
# the network must OUTPUT in logits [-inf, inf]
# make sure input dimension is [B, H, W, C]
def Jindex(target, pred):
    intersection = tf.reduce_sum(target * pred, [0, 1, 2])
    union = tf.reduce_sum(target + pred, [0, 1, 2]) - intersection
    return tf.reduce_mean((intersection + 0.1) / (union + 0.1))


def Dice(target, pred):
    intersection = tf.reduce_sum(2 * pred * target, [0, 1, 2])
    union = tf.reduce_sum(pred + target, [0, 1, 2])
    return tf.reduce_mean((intersection + 0.1) / (union + 0.1))


In [7]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [8]:
# this iteration is calculated fom 160 iteration from
# paper
n_epoch = 20
n_classes = 8
ds = create_ds(1)
test = create_ds(1, False)
model = FCN(8)
# model = sm.Unet(backbone_name='resnet50', encoder_weights='imagenet', encoder_freeze=False, activation='softmax', classes=8)
# model = FCN_ORIG(8)

In [9]:
# pretrain_optimizer = tf.keras.optimizers.Adam()

# lr = 0.001 is good but spiky, next learning rate to test is 0.0005
# both fpn and unet uses 1e-4 learning rate
optimizer = tf.keras.optimizers.Adam(0.000005)
focal_loss = sm.losses.CategoricalFocalLoss()
dice_loss = sm.losses.DiceLoss()

In [10]:
model.build([None, 448, 448, 3])

# model.layers[0].backbone.trainable = False
# for layer in model.layers[0].backbone.layers:
#     layer.trainable = False
#     if layer.name == 'block5a_activation':
#         break

model.compile()
model.summary()

Model: "FCN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Feature_Pyramid_Network (FPN multiple                  5405939   
_________________________________________________________________
up_sampling2d_1 (UpSampling2 multiple                  0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 multiple                  0         
_________________________________________________________________
up_sampling2d_3 (UpSampling2 multiple                  0         
_________________________________________________________________
concatenate (Concatenate)    multiple                  0         
_________________________________________________________________
conv2d_12 (Conv2D)           multiple                  2359808   
_________________________________________________________________
batch_normalization_4 (Batch multiple                  2048    

In [11]:
ckpt = tf.train.Checkpoint(model=model, optimizer=optimizer)
ckptmg = tf.train.CheckpointManager(ckpt, 'test_fcn', 5)
ckptmg.restore_or_initialize()

# Make sure you configure the preprocessing for each model correctly.
## FCN, FPN used Efficient and Unet used Resnet50

1. ## Resnet50 needs input to be range [0, 1]
2. ## Efficientnet need to range to be [0, 255]

In [12]:
# Real training
iteration = 0
test_iteration = 0
prev_loss = 0
prev_iou = 0
test_prev_loss = 0
test_prev_iou = 0

for _ in range(n_epoch):
    for bs_images, bs_labels in ds:
        # REMEMBER HERE
        # Comment below if you are training FPN and ORIG_FCN
        # bs_images = sm.get_preprocessing('resnet')(bs_images)

        with tf.GradientTape() as t:
            output = model(bs_images)
            c_loss = 0.5 * dice_loss(bs_labels, output) + focal_loss(bs_labels, output)

        grad = t.gradient(c_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grad, model.trainable_variables))

        with train_summary_writer.as_default():
            if iteration == 0:
                prev_loss = c_loss
                prev_iou = sm.metrics.iou_score(bs_labels, output)
            else:
                prev_loss = 0.01 * c_loss + (1 - 0.01) * prev_loss
                prev_iou = (
                    0.01 * sm.metrics.iou_score(bs_labels, output)
                    + (1 - 0.01) * prev_iou
                )

            tf.summary.scalar("iou", prev_iou, step=iteration)
            tf.summary.scalar("loss", prev_loss, step=iteration)
            iteration += 1

    for bs_images, bs_labels in test:
        output = model(bs_images, training=False)

        c_loss = 0.5 * dice_loss(bs_labels, output) + focal_loss(bs_labels, output)
        sm.metrics.iou_score(bs_labels, output)

        with test_summary_writer.as_default():
            if test_iteration == 0:
                test_prev_loss = c_loss
                test_prev_iou = sm.metrics.iou_score(bs_labels, output)
            else:
                test_prev_loss = 0.01 * c_loss + (1 - 0.01) * test_prev_loss
                test_prev_iou = (
                    0.01 * sm.metrics.iou_score(bs_labels, output)
                    + (1 - 0.01) * test_prev_iou
                )

            tf.summary.scalar("iou", test_prev_iou, step=iteration)
            tf.summary.scalar("loss", test_prev_loss, step=iteration)
            test_iteration += 1
    ckptmg.save()

# To explain about LABELS

1. Background Clutter (0, 0, 0)
2. Building           (128, 0, 0)
3. Road               (128, 64, 128)
4. Tree               (0, 128, 0)
5. Low Vegetation     (128, 128, 0)
6. Moving Car         (64, 0, 128)
7. Static Car         (192, 0, 192)
8. Human              (64, 64, 0)

In [ ]:
# iou loss
intersection = tf.reduce_sum(out_img * y, [0, 1, 2])
union = tf.reduce_sum(out_img + y, [0, 1, 2]) - intersection
# print(tf.reduce_mean(intersection / union))
(intersection / union).numpy()

In [ ]:
# img_a = tf.random.uniform([1, 5, 5, 3])
# img_b = tf.random.uniform([1, 5, 5, 3])
# img_b = tf.zeros([1, 5, 5, 3])
# img_a = tf.zeros_like(img_b)
# dice_loss = sm.losses.DiceLoss()(img_a, img_b)
# dice_loss_2 = Dice(img_a, img_b)
# print(dice_loss, 1 - dice_loss_2)

In [ ]:
# dice loss
intersection = tf.reduce_sum(2 * out_img * y, [0, 1, 2])
union = tf.reduce_sum(out_img + y, [0, 1, 2])
print(tf.reduce_mean(intersection / union))
intersection / union

In [ ]:
def to_save(images, name):
    for idx in range(images[0].shape[-1]):
        byte = tf.image.encode_jpeg(tf.image.convert_image_dtype(images[0, ..., idx][..., tf.newaxis], tf.uint8))
        tf.io.write_file(f'{name}_{idx}_image.jpeg', byte)

to_save(out_img, 'test')
# to_save(y, 'target')